In [1]:
import os

dir_notebook = os.getcwd()
print(dir_notebook)

/home/david/proyectos4/repositorios/Proyectos-SGBA1/Pruebas/Modelos/precio_peninsula


In [2]:
ruta_trabajo = '../../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_trabajo)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1


In [3]:
import pandas as pd
df_train = pd.read_parquet('data/processed/datos_precio/df_train_lagged.parquet')
df_val = pd.read_parquet('data/processed/datos_precio/df_val_lagged.parquet')

In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import mlflow
import mlflow.lightgbm

In [5]:
from sklearn.preprocessing import StandardScaler

# Select features (excluding datetime, object columns and target)
feature_cols = df_train.select_dtypes(include=['int64', 'float64']).columns
feature_cols = feature_cols.drop('€/kwh')

# Prepare X and y for training
X_train = df_train[feature_cols]
y_train = df_train['€/kwh']

# Prepare X and y for validation
X_val = df_val[feature_cols]
y_val = df_val['€/kwh']

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# LOGEAR EXPERIMENTO

In [6]:
import dagshub
dagshub.init(repo_owner='auditoria.SGBA1', repo_name='Proyectos-SGBA1', mlflow=True)

Accessing as auditoria.SGBA1

Initialized MLflow to track repo "auditoria.SGBA1/Proyectos-SGBA1"

Repository auditoria.SGBA1/Proyectos-SGBA1 initialized!

In [7]:
from sklearn.ensemble import RandomForestRegressor
import mlflow

mlflow.set_experiment("precio/")

with mlflow.start_run(run_name="RandomForest_Model_Run_"):
    # Configurar y entrenar el modelo
    params = {
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Crear y entrenar el modelo
    model = RandomForestRegressor(**params)
    model.fit(X_train_scaled, y_train)
    
    # Hacer predicciones
    y_pred_train = model.predict(X_train_scaled)
    y_pred_val = model.predict(X_val_scaled)
    
    # Calcular métricas
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    train_r2 = r2_score(y_train, y_pred_train)
    val_r2 = r2_score(y_val, y_pred_val)
    
    # Registrar parámetros
    mlflow.log_params(params)
    
    # Registrar métricas
    mlflow.log_metric('train_rmse', train_rmse)
    mlflow.log_metric('val_rmse', val_rmse)
    mlflow.log_metric('train_r2', train_r2)
    mlflow.log_metric('val_r2', val_r2)
    
    # Guardar el modelo
    input_example = X_train_scaled[:5]
    mlflow.sklearn.log_model(model, "random_forest_model", input_example=input_example)
    
    # Imprimir resultados
    print(f'Training RMSE: {train_rmse:.4f}')
    print(f'Validation RMSE: {val_rmse:.4f}')
    print(f'Training R2: {train_r2:.4f}')
    print(f'Validation R2: {val_r2:.4f}')


Training RMSE: 0.0070
Validation RMSE: 0.0526
Training R2: 0.8345
Validation R2: 0.0069
🏃 View run RandomForest_Model_Run_ at: https://dagshub.com/auditoria.SGBA1/Proyectos-SGBA1.mlflow/#/experiments/1/runs/f7d6db184f01409da0ac949d6fbcaaa3
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/Proyectos-SGBA1.mlflow/#/experiments/1
